In [1]:
import pandas as pd
import psycopg2
from psycopg2 import Error
import requests as re
import os
weather_key = os.environ["WETH_API"]
from collections import Counter

In [13]:
connection = psycopg2.connect(user="lhl_student",
                                  password="lhl_student",
                                  host="lhl-data-bootcamp.crzjul5qln0e.ca-central-1.rds.amazonaws.com",
                                  port="5432",
                                  database="mid_term_project")
cursor=connection.cursor()

In [3]:
#this notebook was made to gather weather data for my big dataset and experiment with how to make it work in a model
#also used it to change submission data to right format since modeling notebook was running model fitting

In [14]:
submission_data = "select * from flights_test where fl_date <= '2020-01-07'"
cursor.execute(submission_data)
submission_data = pd.read_sql_query(submission_data,connection)

C:\Users\zain_\AppData\Roaming\Python\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [17]:
submission_data
submission_data.to_csv('submission_data.csv',index=False)

In [18]:
submission_data

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363
1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1150,1320,N,90,1,363
2,2020-01-01,WN,WN,WN,4598,WN,N7885A,4598,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",2020,2130,N,70,1,333
3,2020-01-01,WN,WN,WN,4761,WN,N551WN,4761,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",1340,1455,N,75,1,333
4,2020-01-01,WN,WN,WN,5162,WN,N968WN,5162,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",915,1035,N,80,1,333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150618,2020-01-07,DL,DL_CODESHARE,DL,4813,9E,N605LR,4813,11433,DTW,"Detroit, MI",12478,JFK,"New York, NY",1755,1952,N,117,1,509
150619,2020-01-07,DL,DL_CODESHARE,DL,4814,9E,N919XJ,4814,11996,GSP,"Greer, SC",12953,LGA,"New York, NY",600,759,N,119,1,610
150620,2020-01-07,DL,DL_CODESHARE,DL,4815,9E,N936XJ,4815,10397,ATL,"Atlanta, GA",15919,XNA,"Fayetteville, AR",1715,1816,N,121,1,589
150621,2020-01-07,DL,DL_CODESHARE,DL,4815,9E,N936XJ,4815,15919,XNA,"Fayetteville, AR",10397,ATL,"Atlanta, GA",1851,2145,N,114,1,589


In [6]:
submission_data.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time',
       'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance'],
      dtype='object')

In [3]:
bigdf = pd.read_csv("C:/Users/zain_/LHL/Midterm/mid-term-project-I/fulldf.csv")

In [9]:
city_names = bigdf['origin_city_name']
flight_date = bigdf['fl_date']

In [5]:
weather_key = os.environ["WETH_API"]

In [15]:
jsonlist = []
i=0
try:
    for ele in city_names:
        url = 'http://api.weatherapi.com/v1/current.json'
        params = {'key': weather_key, 'q':ele, 'dt':flight_date[i]}
        response = re.get(url, params = params)
        i+=1
        b = response.json()
        jsonlist.append(b)
    conditions = []
    for ele in jsonlist:
        conditions.append(ele['current']['condition']['text'])
    bigdf['weather_status'] = conditions
except:
    pass

In [20]:
conditions = []
for ele in jsonlist:
    conditions.append(ele['current']['condition']['text'])

In [53]:
Counter(conditions)

Counter({'Sunny': 25148,
         'Partly cloudy': 34447,
         'Overcast': 3180,
         'Clear': 5865,
         'Mist': 753,
         'Light rain': 158,
         'Fog': 24,
         'Patchy rain possible': 11,
         'Moderate or heavy rain with thunder': 64,
         'Heavy rain': 10,
         'Patchy light rain with thunder': 92,
         'Light rain shower': 2,
         'Moderate rain': 1,
         'Thundery outbreaks possible': 8})

In [78]:
#any types with low counts merge with similar category
#make into a function, replace dataframe column with new values
adjusted_conditions = []
conditions = [x.lower() for x in conditions]
for ele in conditions:
    if 'fog' in ele:
        adjusted_conditions.append('mist')
    elif 'rain' in ele:
        adjusted_conditions.append('rain')
    elif 'thunder' in ele:
        adjusted_conditions.append('thunder')
    elif 'snow' in ele:
        adjusted_conditions.append('rain')
    elif 'sunny' in ele:
        adjusted_conditions.append('clear')
    else:
        adjusted_conditions.append(ele)

In [80]:
#data for one hot encoding
set(adjusted_conditions)

{'clear', 'mist', 'overcast', 'partly cloudy', 'rain', 'thunder'}

In [60]:
set(conditions)

{'Clear',
 'Fog',
 'Heavy rain',
 'Light rain',
 'Light rain shower',
 'Mist',
 'Moderate or heavy rain with thunder',
 'Moderate rain',
 'Overcast',
 'Partly cloudy',
 'Patchy light rain with thunder',
 'Patchy rain possible',
 'Sunny',
 'Thundery outbreaks possible'}

In [39]:
Counter(conditions)

Counter({'Sunny': 25148,
         'Partly cloudy': 34447,
         'Overcast': 3180,
         'Clear': 5865,
         'Mist': 753,
         'Light rain': 158,
         'Fog': 24,
         'Patchy rain possible': 11,
         'Moderate or heavy rain with thunder': 64,
         'Heavy rain': 10,
         'Patchy light rain with thunder': 92,
         'Light rain shower': 2,
         'Moderate rain': 1,
         'Thundery outbreaks possible': 8})

In [30]:
weather_df = bigdf[0:69763]

In [31]:
weather_df['weather_condition'] = conditions

C:\Users\zain_\AppData\Local\Temp\ipykernel_8636\2237297297.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_df['weather_condition'] = conditions


In [32]:
weather_df['weather_condition']

0                Sunny
1                Sunny
2        Partly cloudy
3             Overcast
4        Partly cloudy
             ...      
69758            Sunny
69759    Partly cloudy
69760    Partly cloudy
69761            Sunny
69762            Sunny
Name: weather_condition, Length: 69763, dtype: object

In [81]:
main_df = weather_df

In [82]:
main_df.to_csv('main_df.csv',index=False)